In [12]:
import numpy as np
import qilisdk
import qutip as qt
import matplotlib.pyplot as plt
import sys

In [13]:
sys.path.append('/Users/omichel/Desktop/qilimanjaro/projects/retech/retech_2025/src')

import hamiltonian
import dynamics
import plots

In [14]:
# Example usage for Z-basis measurement on a single qubit
z_basis = [qt.basis(2, 0) * qt.basis(2, 0).dag(),  # |0⟩⟨0|
           qt.basis(2, 1) * qt.basis(2, 1).dag()]   # |1⟩⟨1|

In [15]:
def create_random_hamiltonian(Nqubits, min_weight, max_weight):
    H_out = hamiltonian.QutipHamiltonian(Nqubits)
    single_x_weights = np.random.uniform(min_weight, max_weight, size=Nqubits)
    single_z_weights = np.random.uniform(min_weight, max_weight, size=Nqubits)

    interaction_weights = np.random.uniform(min_weight, max_weight, size=int(Nqubits*(Nqubits-1)/2))
    interaction_counter = 0
    for i in range(Nqubits):
        H_out.add_x_field(i, single_x_weights[i])
        H_out.add_z_field(i, single_z_weights[i])

        for j in range(i+1, Nqubits):
            H_out.add_ZZ_term(i, j, interaction_weights[interaction_counter])
            interaction_counter+=1
    
    return H_out

In [16]:
H_test = create_random_hamiltonian(1, -1.0, 1.0)

In [17]:
size = 1

H_fix = hamiltonian.QutipHamiltonian(size)

H_fix.add_z_field(0, 1)
#H_fix.add_z_field(1, 1)
H_fix.add_x_field(0, 0.5)
#H_fix.add_x_field(1, 0.5)
#H_fix.add_ZZ_term(0, 1, 0.25)
print(H_fix.H)

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ 1.   0.5]
 [ 0.5 -1. ]]


In [18]:
psi_0 = dynamics.initialize_down_state(size)

print(psi_0)

Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]]


In [27]:
ti = 0
tf = 2
nsteps = 1000

sim = dynamics.time_evolution(H_fix, psi_0, ti, tf, nsteps)

In [28]:
observables = dynamics.calculate_expectation_values(sim, H_fix)

In [29]:
def calculate_probability(state, qubit_index, basis='Z', shots=10000):
    # Measure first qubit
    outcomes, probs =dynamics.measure_qubit(state, qubit_index, basis, shots)
    counts = np.bincount(outcomes, minlength=2)
    average = (counts[0] * 1 + counts[1] * (-1)) / shots

    return counts, average

In [30]:
print(observables['Sz'][-1])
print(observables['Sx'][-1])

[0.75241098]
[0.49517804]


In [31]:
counts_z, average_z = calculate_probability(sim.states[-1], 0, 'Z', 100000)
counts_x, average_x = calculate_probability(sim.states[-1], 0, 'X', 100000)

print(average_z)
print(average_x)

0.75136
0.49492
